# import


In [1]:
import time
start_time = time.time()

In [2]:
"""
we make recommendation based on the similarity of the items for food recipes based on the ingredients and recipe specifications using the cosine similarity method
and train word2vec model from scratch using gensim library on my dataset and then use the trained model to find the similarity between the ingredients of the recipes.
"""

# Importing the libraries
# !pip install textblob
# !pip install POT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import warnings
# pd.set_option('display.max_colwidth', None)
nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')
nltk.download('wordnet')
# show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# # Importing the dataset
# df = pd.read_csv('https://github.com/eslamahmedamit/food-recipes-recsys/blob/87cf871934c5eb6adb16759ab0d190fba921211a/recipes_data_graduation_project.csv?raw=true')

# # Data Preprocessing
# df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
# import pandas as pd
# # https://github.com/eslamahmedamit/food-recipes-recsys/blob/87cf871934c5eb6adb16759ab0d190fba921211a/recipes_data_graduation_project.csv
# df = pd.read_csv('https://github.com/eslamahmedamit/food-recipes-recsys/blob/87cf871934c5eb6adb16759ab0d190fba921211a/recipes_data_graduation_project.csv?raw=true')
# # df

In [4]:
# https://drive.google.com/file/d/18obaPDPu58OnzhnJgVYLzg1C-XR1ER-4/view
# https://drive.google.com/file/d/1vw6D0GTM8zAg7U_O0PAEwIEw16M-rt9X/view?usp=sharing
!gdown --id "18obaPDPu58OnzhnJgVYLzg1C-XR1ER-4"
!gdown --id "1vw6D0GTM8zAg7U_O0PAEwIEw16M-rt9X"

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=18obaPDPu58OnzhnJgVYLzg1C-XR1ER-4
To: /content/out_text_data.csv
100% 824M/824M [00:06<00:00, 126MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1vw6D0GTM8zAg7U_O0PAEwIEw16M-rt9X
To: /content/word2vec.model
100% 17.1M/17.1M [00:00<00:00, 64.1MB/s]


In [5]:
text_data = pd.read_csv('out_text_data.csv')
word2vec_model = Word2Vec.load("word2vec.model")

# Recommendation for Text Data

### preprocessing

In [6]:
# print text cleaning steps
'''
 text cleaning steps
    1- remove html tags
    2- remove special characters
    3- remove punctuation
    4- remove stopwords
    5- lemmatization
    6- lowercase
    7- remove numbers
    8- remove extra spaces
    9- remove single characters
    10- remove short words

'''
from tqdm.notebook import tqdm


# 1- remove html tags
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# 2- remove special characters
def remove_special_characters(text):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

# 3- remove punctuation
def remove_punctuation(text):
    pattern = r'[^\w\s]'
    text = re.sub(pattern, '', text)
    return text

# 4- remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

# 5- lemmatization
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_tokens])
    return lemmatized_output

# 6- lowercase
def lowercase(text):
    return text.lower()

# 7- remove numbers
def remove_numbers(text):
    pattern = r'[0-9]'
    text = re.sub(pattern, '', text)
    return text

# 8- remove extra spaces
def remove_extra_spaces(text):
    pattern = r'\s+'
    text = re.sub(pattern, ' ', text)
    return text

# 9- remove single characters
def remove_single_characters(text):
    pattern = r'\s+[a-zA-Z]\s+'
    text = re.sub(pattern, ' ', text)
    return text

# 10- remove short words
def remove_short_words(text):
    pattern = r'\W*\b\w{1,3}\b'
    text = re.sub(pattern, '', text)
    return text

# 11 - conver plural to singular using textblob library
def singular(text):
    from textblob import TextBlob
    text = TextBlob(text)
    text = text.words.singularize()
    return text

# 12 - remove most 3 common words ['tablespoon','teaspoon','cup']
def remove_common_words(text):
    common_words = ['tablespoon','teaspoon','cup']
    text = ' '.join([word for word in text.split() if word not in common_words])
    return text


# apply all text cleaning steps
def text_cleaning(text):
    text = remove_html_tags(text)
    text = remove_special_characters(text)
    text = remove_punctuation(text)
    # print(text)
    text = remove_stopwords(text)
    text = lemmatization(text)
    text = lowercase(text)
    text = remove_numbers(text)
    text = remove_extra_spaces(text)
    text = remove_single_characters(text)
    # text = remove_short_words(text)
    # add text to string
    text = remove_common_words(text)
    text = singular(text)
    text = ' '.join(text)

    try:
      pbar.update(1)
    except:
      pass
    return text

# apply text cleaning steps on text data
# for feature in text_features:
#     text_data[feature] = text_data[feature].apply(text_cleaning)


# create function to apply with progress bar
def apply_with_bar(org_data,column):
    global pbar

    pbar = tqdm(total=len(org_data))

    org_data[f'{column}_cleaned'] = org_data[column].apply(text_cleaning)
    print(f'Done cleaning {column}')
    pbar.close()

# # apply function to all text columns
# clean_features = ['summary']
# clean_features = ['name','summary','ingredients']
# clean_features = ['name', 'category', 'summary', 'ingredients','diet_type']
# for column in clean_features:
#     apply_with_bar(text_data,column)

## Recommend Function

In [7]:
# get similarity between two ingredients with different teqnique
# convert the ingredients to vector if the ingredients is in the vocabulary using get_vector_pre_trained function
# edit the function to split words
def get_vector_pre_trained(x,model):
    s_vec = w_vec = np.zeros(model.vector_size)

    for word in x.split():
        if word in model:
            w_vec = model[word]
            s_vec = s_vec + w_vec
    # if s_vec.all() == 0:
    #     s_vec = np.zeros(256)
    return s_vec


In [8]:
def edit_vectors(vector_str):
    # Use regular expressions to convert str to float
    vector_str = vector_str.replace("[[]]", '')
    vector_str = re.sub(r"\n", "", vector_str)
    vector_str = re.sub(r"\[", "", vector_str)
    vector_str = re.sub(r"\]", "", vector_str)

    vector_str = re.sub(r"  ", " ", vector_str)
    vector_str = vector_str.split(' ')

    vector_str = [float(element) for element in vector_str if element != '']

    return vector_str

# type(edit_vectors(text_data['summary_vector'][4]))#[:20]

	# ingredients_vector	name_vector	category_vector	summary_vector	diet_type_vector	all_text_vector

text_data['ingredients_vector'] = text_data['ingredients_vector'].apply(edit_vectors)
text_data['name_vector'] = text_data['name_vector'].apply(edit_vectors)
text_data['category_vector'] = text_data['category_vector'].apply(edit_vectors)
text_data['summary_vector'] = text_data['summary_vector'].apply(edit_vectors)
text_data['diet_type_vector'] = text_data['diet_type_vector'].apply(edit_vectors)
text_data['all_text_vector'] = text_data['all_text_vector'].apply(edit_vectors)

In [9]:
# now we need to build a recommender system based on the ingredients, name, category, summary, diet_type to predict the best recipe for the user with high similarity
# we will use cosine similarity to get the similarity between the user input and the recipes in the dataset
# we will use the ingredients, name, category, summary, diet_type as features to predict the best recipe for the user

def get_similar_recipe_full_model(test_df_input,no_of_recipes=5):
    clean_features = ['name', 'category', 'summary', 'ingredients','diet_type']
    for column in clean_features:
        # apply_with_bar(test_df_input,column)
        test_df_input[f'{column}_cleaned'] = test_df_input[column].apply(text_cleaning)

    # get the vector of the input
    ingredients_vector = get_vector_pre_trained(test_df_input['ingredients_cleaned'][0],word2vec_model.wv)
    name_vector = get_vector_pre_trained(test_df_input['name_cleaned'][0],word2vec_model.wv)
    category_vector = get_vector_pre_trained(test_df_input['category_cleaned'][0],word2vec_model.wv)
    summary_vector = get_vector_pre_trained(test_df_input['summary_cleaned'][0],word2vec_model.wv)
    diet_type_vector = get_vector_pre_trained(test_df_input['diet_type_cleaned'][0],word2vec_model.wv)

    # try to abbreviation the previous code
    # get the vector of the input


    # get the cosine similarity between the input and the recipes in the dataset
    text_data['ingredients_similarity'] = text_data['ingredients_vector'].apply(lambda x: word2vec_model.wv.cosine_similarities(ingredients_vector,[x])[0])
    text_data['name_similarity'] = text_data['name_vector'].apply(lambda x: word2vec_model.wv.cosine_similarities(name_vector,[x])[0])
    text_data['category_similarity'] = text_data['category_vector'].apply(lambda x: word2vec_model.wv.cosine_similarities(category_vector,[x])[0])
    text_data['summary_similarity'] = text_data['summary_vector'].apply(lambda x: word2vec_model.wv.cosine_similarities(summary_vector,[x])[0])
    text_data['diet_type_similarity'] = text_data['diet_type_vector'].apply(lambda x: word2vec_model.wv.cosine_similarities(diet_type_vector,[x])[0])
    # get the mean of the similarity between the input and the recipes in the dataset
    text_data['mean_similarity'] = (text_data['ingredients_similarity'] + text_data['name_similarity'] + text_data['category_similarity'] + text_data['summary_similarity'] + text_data['diet_type_similarity'])/5
    # get the top 5 recipes with high similarity

    top_recipes = text_data.sort_values(by='mean_similarity',ascending=False).head(no_of_recipes)
    return top_recipes

# test the function
test_name = 'Macaroni and Cheese'
test_category = 'main-dish'
test_summary = 'This is a recipe for macaroni and cheese.'
test_ingredients = 'macaroni, cheese, butter, milk'
test_diet_type = 'Vegetarian'

# add to dataframe

test_df = pd.DataFrame({'name':[test_name],'category':[test_category],'summary':[test_summary],'ingredients':[test_ingredients],'diet_type':[test_diet_type]})

# get_similar_recipe_full_model(test_df)

In [10]:
# make a function to get the top 5 recipes with high similarity but similar with free text input and with all text column
def get_similar_recipe_free_text(test_text_input,no_of_recipes=5):
    # get the vector of the input
    text_vector = get_vector_pre_trained(test_text_input,word2vec_model.wv)
    # get the cosine similarity between the input and the recipes in the dataset
    text_data['text_similarity'] = text_data['all_text_vector'].apply(lambda x: word2vec_model.wv.cosine_similarities(text_vector,[x])[0])
    # get the top 5 recipes with high similarity
    top_recipes = text_data.sort_values(by='text_similarity',ascending=False).head(no_of_recipes)
    return top_recipes


# test the function
test_text = 'This is a recipe for macaroni and cheese.'
# get_similar_recipe_free_text(test_text)

In [11]:
# Make final function merge the two functions together based on mode parameter to get free text mode or not full model
# make the function input as a dictionary
def get_similar_recipe(user_input):
    if user_input['mode']== 'full_model':
        # convert to dataframe from dictionary to dataframe using keys as columns
        user_input_df = pd.DataFrame(test_input,index=[0])
        # user_input_df = pd.DataFrame(user_input['name'],user_input['category'],user_input['summary'],user_input['ingredients'],user_input['diet_type'])

        return get_similar_recipe_full_model(user_input_df,user_input['no_of_recipes'])
    elif user_input['mode'] == 'free_text':
        return get_similar_recipe_free_text(user_input['text'],user_input['no_of_recipes'])
    else:
        return 'Please enter the correct mode'


# test the function
test_input = {'name':'Macaroni and Cheese','category':'main-dish','summary':'This is a recipe for macaroni and cheese.',
              'ingredients':'macaroni, cheese, butter, milk','diet_type':'Vegetarian',
              'mode':'full_model','no_of_recipes':5}


get_similar_recipe(test_input)

# test the function with free text mode
test_input = {'text':'This is a recipe for macaroni and cheese.','mode':'free_text','no_of_recipes':5}
get_similar_recipe(test_input)


,Unnamed: 0,url,name,category,summary,ingredients,diet_type,allergens,cuisine,name_cleaned,category_cleaned,summary_cleaned,ingredients_cleaned,diet_type_cleaned,all_text,ingredients_vector,name_vector,category_vector,summary_vector,diet_type_vector,all_text_vector,ingredients_similarity,name_similarity,category_similarity,summary_similarity,diet_type_similarity,mean_similarity,text_similarity
17093,17093,https://www.allrecipes.com/recipe/237064/chef-...,Chef John's Homemade Cheese,world-cuisine,Usually making something like fresh cheese at ...,1 quart whole milk ; 1 cup cultured buttermilk...,"vegetarian, omnivore, keto",milk,chinese,chef john homemade cheese,worldcuisine,usually making something like fresh cheese hom...,quart whole milk cultured buttermilk lemon jui...,vegetarian omnivore keto,chef john homemade cheese quart whole milk cul...,"[-0.19837799, -0.06539053, 0.30719837, -0.1399...","[-0.23810511, 0.08556822, 0.05909911, 0.060571...","[0.09202719, -0.0477601, -0.12202831, 0.026606...","[-1.25981323, 0.42215108, -1.39175358, 0.52366...","[0.01124892, 0.06244419, -0.08738073, -0.03586...","[-1.5930202, 0.45701287, -1.23486513, 0.435019...",0.280709,0.089196,0.674928,0.195105,0.331145,0.314217,0.398472
24911,24911,https://www.allrecipes.com/recipe/15343/menudo/,Menudo,soups-stews-and-chili,This dish depends on how much of each ingredie...,"2 pounds beef tripe ; 2 onions, chopped ; 4 (...",omnivore,no_allergens,chinese,menudo,soupsstewsandchili,thi dish depend much ingredient put it depend ...,pound beef tripe onion chopped ounce can white...,omnivore,menudo pound beef tripe onion chopped ounce ca...,"[-0.22396242, -0.05157039, 0.50392556, 0.06134...","[-0.06606941, 0.05903533, 0.11464968, -0.11381...","[0.10139953, -0.07426593, 0.00211118, 0.048104...","[-0.5400863, 1.1556715, -0.75533601, 1.2593419...","[0.02420655, 0.05769445, -0.05357387, -0.07366...","[-0.70451205, 1.14656496, -0.18822345, 1.18131...",0.221661,-0.023317,0.639592,0.083276,0.000451,0.184333,0.382612
12759,12759,https://www.allrecipes.com/recipe/168646/aunti...,Auntie Anita's Lobster Stew,soups-stews-and-chili,Stirring is the most important thing in this m...,2 (1 pound) Maine lobsters ; ½ cup butter ; 1 ...,"vegetarian, omnivore, keto","milk, crustacean_shellfish",unknown,auntie anita lobster stew,soupsstewsandchili,stirring important thing masterpiece otherwise...,pound maine lobster butter quart whole milk,vegetarian omnivore keto,auntie anita lobster stew pound maine lobster ...,"[-0.21986135, -0.1388417, 0.08782941, 0.123600...","[-0.0306272, 0.01226422, 0.01496476, 0.0333516...","[0.10139953, -0.07426593, 0.00211118, 0.048104...","[-0.50716522, 1.4525462, -1.86576846, 0.747241...","[0.01124892, 0.06244419, -0.08738073, -0.03586...","[-0.64500532, 1.31414699, -1.84824383, 0.91643...",0.386301,0.113893,0.639592,0.022767,0.331145,0.298740,0.374409
23658,23658,https://www.allrecipes.com/recipe/138016/brill...,Brilliant Sauteed Broccoli,side-dish,If you want a slightly original restaurant-qua...,1 pound broccoli florets ; 3 tablespoons finel...,"vegetarian, omnivore, paleo",milk,chinese,brilliant sauteed broccoli,sidedish,if want slightly original restaurantquality br...,pound broccoli floret finely grated parmesan c...,vegetarian omnivore paleo,brilliant sauteed broccoli pound broccoli flor...,"[0.05009192, -0.90001818, 0.87432666, -0.15795...","[0.05286979, -0.16882054, 0.07388257, 0.039960...","[0.08671039, -0.08591148, -0.12258501, 0.04769...","[-0.06667369, 0.29832426, -0.9436295, 0.032158...","[0.17145596, -0.03337804, -0.31693207, -0.1072...","[0.29445437, -0.88980398, -0.43493735, -0.1453...",0.269443,0.013735,0.668722,0.062835,0.482987,0.299545,0.374024
343,343,https://www.allrecipes.com/recipe/274411/bucat...,Bucatini Cacio e Pepe (Roman Sheep Herder's Pa...,world-cuisine,This peasant food comes from Roman sheep herde...,1 teaspoon salt ; 1 pound bucatini (dry) ; 2 c...,"vegetarian, omnivore, keto",milk,chinese,bucatini cacio pepe roman s

In [12]:
print("--- %s seconds ---" % (time.time() - start_time))


--- 90.48839235305786 seconds ---


In [13]:
# !pip install flask
# !pip install pyngrok


In [14]:
# import sys
# from pyngrok import ngrok
# from flask import Flask, request, jsonify
# import os

# app = Flask(__name__)
# # make app config "ENV" = "development" to make the app run in development mode
# app.config["ENV"] = "development"
# app.config["USE_NGROK"] = True


# @app.route('/get-recipe', methods=['POST'])
# def get_recipe():

#     json_data = request.get_json()  # Get the JSON data from the request
#     print(json_data)

#     # test_input = json_data['test_input']  # Extract the required input from the JSON
#     # print(test_input)
#     result = get_similar_recipe(json_data).to_json()  # Call your function with the input
#     print("result", jsonify(result)  )
#     return jsonify(result)  # Return the result as a JSON response

# if __name__ == '__main__':
#     app.run()

In [15]:
# if __name__ == '__main__':
#     app.run(host="localhost",port=13950, debug=False)